# Data handling

(howto-data-handling-to-gridded)=
## Converting dataset coordinates from flat list to grid

By default measurement control returnds data in a sparse format.
The resulting dataset has a single dimension, and all settables and gettables share it, for example:

In [ ]:
from pathlib import Path

import numpy as np
from qcodes import ManualParameter, Parameter, validators

from quantify_core.data.handling import set_datadir, to_gridded_dataset
from quantify_core.measurement import MeasurementControl

set_datadir(Path.home() / "quantify-data")

time_a = ManualParameter(
    name="time_a",
    label="Time A",
    unit="s",
    vals=validators.Numbers(),
    initial_value=1,
)
time_b = ManualParameter(
    name="time_b",
    label="Time B",
    unit="s",
    vals=validators.Numbers(),
    initial_value=1,
)
signal = Parameter(
    name="sig_a",
    label="Signal A",
    unit="V",
    get_cmd=lambda: np.exp(time_a()) + 0.5 * np.exp(time_b()),
)

meas_ctrl = MeasurementControl("meas_ctrl")
meas_ctrl.settables([time_a, time_b])
meas_ctrl.gettables(signal)
meas_ctrl.setpoints_grid([np.linspace(0, 5, 10), np.linspace(5, 0, 12)])
dset = meas_ctrl.run("2D-single-float-valued-settable-gettable")
dset

This format is very close to {func}`COO sparse matrix <scipy.sparse.coo_matrix>`, except that coordinates are not integer.

If the initial data is gridded, it is more convenient to resotre this structure in the dataset for processing.
There is a utility function {func}`quantify_core.data.handling.to_gridded_dataset` for that:

In [ ]:
dset_grid = to_gridded_dataset(dset)
dset_grid

We see that now dataset has two dimensions (`x0` and `x1`), that represent the initial grid for settables `time_a` and `time_b`.
Frecuently this simplifies data processing and analysis a lot, for example, we can display the data straight away:

In [ ]:
dset_grid.y0.plot(cmap="viridis")